In [ ]:
import sys
import pandas as pd
import numpy as np
import CAPS
from CAPS import ProgressBar
import pickle
from sklearn.model_selection import train_test_split, KFold
from tpot import TPOTRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import os.path
from random import sample, seed
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf

data_path = ["..", "data"]

# Data Cleaning

In [ ]:
dropped = pd.DataFrame(columns=['Why', 'How Many'])

user_input = input("Skip creating master_frame1.csv?(Y/n) ")
# Since creating master_frame1 can take a not insignificant amount of time
# this line is included so that user may skip doing this.
if user_input != 'Y':

    # Cleaning of Appointments data frame
    apts_df = pd.read_csv(os.path.join(*data_path, "Appointments.csv"), low_memory=False)

    rows = apts_df.shape[0]
    dropped.loc[dropped.shape[0], :] = ('Rows in Appointments.csv', rows)  # 311168

    # Fix ClientIDS - data is useless if we are missing them
    apts_df = apts_df.replace('#NULL!', np.nan)
    apts_df.dropna(subset=["ClientID"], inplace=True)
    apts_df["ClientID"] = apts_df["ClientID"].astype(float).astype(int).astype(str)

    dropped.loc[dropped.shape[0], :] = ('NA in ClientID', apts_df.shape[0] - rows)

    # Hide away 20% of Clients for final validation
    seed(34)
    hide = sample(sorted(apts_df['ClientID'].unique()), int(len(apts_df['ClientID'].unique()) * .2))
    apts_df = apts_df[~apts_df['ClientID'].isin(hide)]
    pd.Series(hide).to_csv(os.path.join(*data_path, 'ClientsHiddenAway'), index=False)

    rows = apts_df.shape[0]
    dropped.loc[dropped.shape[0], :] = ('Rows after hiding 20% of clients', rows)

    # Make Dates a Datetime object
    apts_df['Date'] = pd.to_datetime(apts_df['Date'])

    apts_df = apts_df[
        (apts_df["AttendanceDescription"] == 'Attended') | (
            apts_df['AttendanceDescription'] == 'Client No Show')
    ]
    dropped.loc[dropped.shape[0], :] = ('Neither "Attended" nor "Client No Show"', apts_df.shape[0] - rows)
    rows = apts_df.shape[0]

    # Drop some appointment types that Dr. Erekson said were not of interest
    apts_df = apts_df[~apts_df["AppType"].isin(['Bio Individual',
                                                'Bio Intake',
                                                'Medication Re-check',
                                                'Medication Initial',
                                                'Mindfulness Meditation',
                                                'Bio Class Assignment',
                                                'Phone',
                                                'Testing',
                                                'Biofeedback Training',
                                                'Email',
                                                'Biofeedback Class Assignments',
                                                'Biofeedback Research',
                                                'Group Note',
                                                'Supervision'
                                                ])
                      ]

    dropped.loc[dropped.shape[0], :] = ('Remove certain appointment types', apts_df.shape[0] - rows)

    # Make TherapistID as string
    apts_df["TherapistID"] = apts_df["TherapistID"].astype(str)
    # print(apts_df["TherapistID"].unique())  # No null values

    # ### Aggregate Sessions

    pat_df = pd.read_csv(os.path.join(*data_path, "PatientInformation.csv"), low_memory=False)
    client_IDs = pat_df.loc[~pat_df["ClientID"].isin(hide), 'ClientID'].unique().astype(str)

    def attend(x):
        x = list(x)
        if x[-1] == 'Attended':
            return 1
        else:
            return 0

    n = len(client_IDs)

    master_df = pd.DataFrame(
        columns=['PatientID', 'TherapistID', 'StartDate', 'EndDate',
                 'NumOfAttended', 'Crisis', 'LastAppShowed', 'AttendRate', 'OneAppIsIntake']
    )  # Empty data-frame

    print('Aggregating Sessions')
    for j, client in enumerate(client_IDs):
        ProgressBar(j, n)

        # Data frame containing the AppIDs TherapistIDs, and Dates for a particular client
        app = apts_df[
            apts_df['ClientID'] == client
        ][['AppID', 'TherapistID', 'Date', 'AppType', 'AttendanceDescription']]

        # Sorts the data frame based on date
        app.sort_values('Date', inplace=True)

        if app.shape[0] < 1:
            # If the data-frame has no appointments, it is not included
            continue

        # Create column 'Elapsed' which has the number of days since the last attended appointment
        # of the same therapist for attended appointments and days since last appointment of the
        # same therapist for no-show appointments
        app['Elapsed'] = (
            app.groupby(['TherapistID', 'AttendanceDescription'])['Date'].diff()
        ).apply(lambda x: x.days)

        app.loc[app['AttendanceDescription'] == 'Client No Show', 'Elapsed'] = app.groupby(['TherapistID'])[
            'Date'].diff().apply(lambda x: x.days)

        app.fillna(0, inplace=True)

        # If the appointment of type crisis, yes(1)/no(0)
        app['Crisis'] = app['AppType'].apply(lambda x: 1 if x.lower()[-6:] == 'crisis' else 0)

        app['Session'] = 0  # Assigns zeros to, while also creating, a column called Session
        sess_num = 0

        # Number sessions
        for therapist in app['TherapistID'].unique():
            sessions = []
            # Within a therapist if the time between appointments exceeds 180 days, new session begins
            for i in range(app.loc[app['TherapistID'] == therapist, :].shape[0] - 1):
                sessions.append(sess_num)
                if app.loc[app['TherapistID'] == therapist, :].iloc[i + 1]['Elapsed'] > 180:
                    sess_num += 1
            sessions.append(sess_num)
            app.loc[app['TherapistID'] == therapist, 'Session'] += sessions
            sess_num += 1

        # Makes a data-frame with columns for the start and end dates
        temp = pd.DataFrame(list(
            app.groupby('Session')['Date'].apply(
                lambda x: [min(x), max(x)])
        ), columns=['StartDate', 'EndDate'])

        temp.insert(loc=0, column='PatientID', value=client)
        temp.insert(loc=1, column='TherapistID', value=app.groupby('Session')["TherapistID"].apply(lambda x: x.iloc[0])
                    )  # Adds the therapist ID that corresponds to the therapist during the session dates
        temp['NumOfAttended'] = (
            app[app['AttendanceDescription'] == 'Attended'].groupby(
                'Session')["AppID"].apply(lambda x: len(x))
        )  # Adds number of appointments in session
        temp['Crisis'] = (
            app.groupby('Session')["Crisis"].apply(lambda x: max(x))
        )  # 1 if a crisis appointment type is part of the session, 0 if not
        temp['LastAppShowed'] = (
            app.groupby('Session')['AttendanceDescription'].apply(attend)
        )  # 1 if the client showed up to the last appointment in that session, 0 if not
        temp['AttendRate'] = (
            app.groupby('Session')['AttendanceDescription'].apply(
                lambda x: np.mean(x == 'Attended'))
        )

        # Some client meet a particular therapist once for intake but not again
        # This OneAppIsIntake column is so that we can drop them later
        temp['OneAppIsIntake'] = app.groupby('Session')['AppType'].apply(
            lambda x: 1 if (len(x) == 1) and (list(x)[0][-6:].lower() == 'intake') else 0
        )

        master_df = pd.concat([master_df, temp])

    master_df['NumOfAttended'].replace({np.nan: 0}, inplace=True)
    master_df['StartDate'] = pd.to_datetime(master_df['StartDate'])
    master_df['EndDate'] = pd.to_datetime(master_df['EndDate'])
    rows = master_df.shape[0]
    dropped.loc[dropped.shape[0], :] = ('Number of Sessions', rows)
    master_df = master_df[master_df['OneAppIsIntake'] != 1].drop(
        'OneAppIsIntake', axis=1)
    dropped.loc[dropped.shape[0], :] = ('Drop Sessions that are solely Intake', master_df.shape[0] - rows)
    master_df.to_csv(os.path.join(*data_path, 'master_frame1.csv'), index=False)
    print('master_frame1.csv created')

    master_df.index = range(master_df.shape[0])

# End: if user_input != 'Y':

# ### Match patient information to session
print('Matching patient information to session')

pat_df = pd.read_csv(os.path.join(*data_path, "PatientInformation.csv"), low_memory=False)
pat_df['ClientID'] = pat_df['ClientID'].astype(float).astype(int).astype(str)
pat_df['notedate'] = pd.to_datetime(pat_df['notedate'])
key = pd.read_csv(os.path.join(*data_path, 'Key.csv'))

# Rename columns so that they are not codes but descriptions
col = list(pat_df.columns)
for j, i in enumerate(col):
    i = i.upper()
    if i in list(key['Column Name']):
        col[j] = str(key[key['Column Name'] == i].Description.values[0])
pat_df.columns = col


# Drop columns marked as OBSOLETE
drop = [i for i in pat_df.columns if 'OBSOLETE' in i]
pat_df.drop(drop, axis=1, inplace=True)

# Oddly, the non-inactive Gender is all NA values
pat_df['Gender'] = pat_df['Gender <inactive>']

# Drop columns marked as inactive
drop = [i for i in pat_df.columns if 'inactive' in i]
pat_df.drop(drop, axis=1, inplace=True)

# Subset to columns that Dr. Davey Erekson suggested
data_dict = pd.read_csv(os.path.join(*data_path, 'DataDictionary.csv'))

keep = ['ClientID', 'notedate']
keep.extend(data_dict.loc[data_dict['DrEreksonRecommended'].isin(
    ['yes', 'maybe']), 'PatientInfoColumn'].values)
pat_df = pat_df[keep]


# Replace <No Response> with Nan for uniformity
pat_df.replace('<No Response>', np.nan, inplace=True)

# NA in age
pat_df['age'] = pd.to_numeric(pat_df['age'], errors='coerce')
# Tyler Mansfield's cleaning said CAPS does not have clients under 16
pat_df.loc[pat_df['age'] <= 16, 'age'] = np.nan

# Drop the rows that lack the client-answered part of the information
subset_pat = pat_df.copy().loc[
    ~pat_df.loc[:, 'Confusion about religious beliefs or values':].isna().apply(lambda x: all(x), axis=1), :
]
# Some rows have a non-NA while the rest of the answers were not filled out,
# but this might be caused by how the information was encoded, not because
# the client answered only those questions
# So that is why it looks at the columns after 'Confusion about religious beliefs or values'

master_df = pd.read_csv(os.path.join(*data_path, 'master_frame1.csv'))
# Convert columns to certian types
master_df['PatientID'] = master_df['PatientID'].astype(str)
master_df['StartDate'] = pd.to_datetime(master_df['StartDate'])
master_df['EndDate'] = pd.to_datetime(master_df['EndDate'])
subset_pat['ClientID'] = subset_pat['ClientID'].astype(str)
subset_pat['notedate'] = pd.to_datetime(subset_pat['notedate'])

# Add columns to master_df that are in pat_df, except for ClientID
add_columns = pd.DataFrame(columns=subset_pat.drop('ClientID', axis=1).columns)
master_df = master_df.join(add_columns)


stable = ['Gender', 'Race / Ethnicity', 'International Student',
          "Have you been diagnosed with an autism-spectrum disorder or Asperger's Syndrome?",
          'First Generation', 'Financial Stress Past', 'Religion', 'Religion Importance']

last_time = [c for c in pat_df.columns if 'Last time' in c]

changing = list(set(pat_df.loc[:, 'Gender':].columns) - set(stable) - set('age') - set(last_time))

convert_dictionary = {'Within the last year': 365, 'More than 5 years ago': 'same',
                      'Within the last 1-5 years': 'same', 'Within the last 2 weeks': 14,
                      'Within the last month': 30, 'Never': 'same'}


def last_time_convert(value, time_past):
    x = convert_dictionary[value] + time_past
    if x <= 14:
        return 'Within the last 2 weeks'
    if x <= 30:
        return 'Within the last month'
    if x <= 365:
        return 'Within the last year'
    if x <= 1825:
        return 'Within the last 1-5 years'
    return 'More than 5 years ago'

lost = []
session_survey = {'master_frame1 row': ['ClientID', 'StartDate - MatchedSurvey', 'dict(StartDate, notedate)']}
session_survey_extra = []

for i in range(master_df.shape[0]):
    ProgressBar(i, master_df.shape[0])
    # Get a session (row) of master_df
    client = master_df.loc[i]
    # Find the patient information for that client
    information = subset_pat[subset_pat['ClientID'] == client['PatientID']].sort_values('notedate')

    if information.shape[0] == 0:
        lost.append(['NoPaper', client['PatientID']])
        session_survey_extra.append([client['PatientID'], np.nan, np.nan, np.nan, np.nan])
        # If there is not information, then add to lost list so that you may look at these clients
        continue

    # Calculate how difference between session start and survey notedate 
    # and make it the information index
    information.index = (client['StartDate'] - information['notedate']
                         ).apply(lambda x: x.days)  # if notedate is after StartDate, returns a negative value

    information.sort_index()  # low to high

    # Take the nearst survey to assign for patient information of that session in master_df
    try:
        survey = information.loc[
            (information.index >= 0) & (information.index <= 90), :].iloc[0]  # First tries to find a survey 90 days before
        # the session started
    except IndexError:
        survey = information.iloc[np.argmin(abs(information.index)), :]
        # if try fails, then use closest (absolute) survey

    # add the matched survey index and all the information indexs so that we may look at them
    session_survey[i] = [client['PatientID'], survey.name, {'StartDate':client['StartDate'], 'notedate': information['notedate'].values}]

    good_match = (survey.name >= 0) & (survey.name <= 180)
    session_survey_extra.append([client['PatientID'],
                                 client['TherapistID'],
                                 client['StartDate'],
                                 survey['notedate'],
                                 good_match])
    # end of ss_extra

    # Fill missing values using other surveys
    if information.shape[0] > 1:
        for c in stable:
            if np.isnan(survey.isna()[c]):
                for i in information.drop(survey.index).index:
                    if ~np.isnan(information.isna().loc[i, c]):
                        survey[c] = information.loc[i, c].values
                        break

        # For changing, look only at the surveys that were within
        # 90 days before the session start
        for c in changing:
            if np.isnan(survey.isna()[c]):
                for i in information.drop(survey.index).index:
                    if (~np.isnan(information.isna().loc[i, c])
                        ) & (information.loc[i].index <= 90
                             ) & (information.loc[i].index >= 0):
                        survey[c] = information.loc[i, c].values
                        break

        for c in last_time:
            if np.isnan(survey.isna()[c]):
                for i in information.drop(survey.index).index:
                    if (~np.isnan(information.isna().loc[i, c])) & (information.loc[i].index >= 0):
                        value = information.loc[i, c].values
                        time_past = information.loc[i].index
                        fill = value
                        if convert_dictionary[value] != 'same':
                            # Later, answers that are beyond one year will be changed to a 0,
                            # so not it does not need to be changed if it already beyond a year
                            fill = last_time_convert(value, time_past)

                        survey[c] = fill
                        break

        if np.isnan(survey.isna()['age']):
            for i in information.drop(survey.index).index:
                if (~np.isnan(information.isna().loc[i, c])
                    ) & (information.loc[i].index <= 90
                         ) & (information.loc[i].index >= 0):
                    age = information.loc[i, c].values
                    fill_age = age + ((survey.index - information.loc[i].index) / 365.25)
                    survey['age'] = fill_age
                    break

    master_df.loc[i, subset_pat.drop('ClientID', axis=1).columns] = survey

# End of session-survey matching

rows = master_df.shape[0]
master_df.dropna(subset='notedate', inplace=True)  # Drop sessions that had no surveys
dropped.loc[dropped.shape[0], :] = ('No surveys', rows - master_df.shape[0])

master_df.index = range(master_df.shape[0])

master_df['notedate'] = pd.to_datetime(master_df['notedate'])

# New column DateDifference that is the difference between the session start date and the notedate
date_diff = master_df[['StartDate', 'notedate']].apply(
    lambda x: (x['StartDate'] - x['notedate']).days, axis=1)
master_df.insert(loc=master_df.columns.get_loc('notedate') + 1,
                 column='DateDifference', value=date_diff)

# Update age
master_df['age'] = master_df['age'].astype(float) + (master_df['DateDifference'] / 365)

# ### To Numeric

# Answers on a scale are converted to numbers
scale1 = {'Never': 0, '1 time': 1, '2-3 times': 2, '4-5 times': 3,
          'More than 5 times': 4}
scale2 = {'Never stressful': 0, 'Rarely stressful': 1, 'Sometimes stressful': 2,
          'Often stressful': 3, 'Always stressful': 4}
scale3 = {'0': 0, 'Rarely (0-1 nights)': 0, 'Sometimes (2-3 nights)': 1, 'Often (4+ nights)': 2}
scale4 = {'Strongly disagree': -2, 'Somewhat disagree': -1,
          'Neutral': 0, 'Somewhat agree': 1, 'Strongly agree': 2}
scale5 = {'Very unimportant': -2, 'Unimportant': -1,
          'Neutral': 0, 'Important': 1, 'Very important': 2}
scale_list = [scale1, scale2, scale3, scale4, scale5]

# Make replace dictionary
replace = {}
for c in master_df.columns:
    unique_v = master_df[c].dropna().unique()
    for scale in scale_list:
        if set(scale.keys()).issuperset(unique_v):
            replace[c] = scale
            break

master_df.replace(replace, inplace=True)


last_time = [c for c in master_df.columns if 'Last time' in c]
for c in last_time:
    master_df[c].replace({'Within the last 2 weeks': 1, 'Within the last month': 1,
                          'Within the last year': 1, 'Within the last 1-5 years': 0,
                          'More than 5 years ago': 0, 'Never': 0, '<No Response>': 0}, inplace=True)
# 1 if more recent than a year
# Seth set this somewhat arbitrarily, so it may be good
# to ask Dr. Erekson if the year threshold is the most relevent of the options to clinicians

rename_1 = {}
for c in last_time:
    master_df[c] = master_df[c].astype(float)
    rename_1[c] = c.replace('Last time', 'Recent')

    # Add to ModifiedName
    data_dict.loc[data_dict['PatientInfoColumn'] == c,
                  'ModifiedName'] = c.replace('Last time', 'Recent')
    # End of adding

master_df.rename(columns=rename_1, inplace=True)

academic_columns = pd.get_dummies(master_df['Academic Status'], prefix='', prefix_sep='')
academic_other = ['Non-student', 'Other (please specify)', 'Non-degree student',
                  'Faculty or staff', 'High-school student taking college classes']

for c in academic_columns.drop(academic_other, axis=1).columns:
    master_df.insert(loc=master_df.columns.get_loc(
        'Academic Status'), column=c, value=academic_columns[c])
master_df.drop('Academic Status', axis=1, inplace=True)

modified_names = list(academic_columns.drop(academic_other, axis=1).columns)

# Add to ModifiedName
for i in range(len(modified_names)):
    index = data_dict.loc[data_dict['PatientInfoColumn'] == 'Academic Status'].index[0] + i
    if data_dict.at[index, 'ModifiedName'] != modified_names[i]:
        data_dict.loc[index, 'ModifiedName'] = modified_names[i]
# End of adding


# Convert certain columns to yes(1)/no(1)

for c in ['Prior Counseling', 'Prior Meds']:
    master_df[c] = master_df[c].apply(lambda x: 0 if x == 'Never' else np.nan if pd.isna(x) else 1)

rename_2 = {'Gender': 'Female',
            'Relationship Status': 'MarriedMale',
            'Race / Ethnicity': 'RacialMinority',
            'Sexual Orientation': 'SexOrientationMinority',
            'Religion': 'ReligiousMinority',
            'In what college is your current major?': 'NursingOrLaw',
            'Housing Other': 'Homeless'}

master_df.rename(columns=rename_2, inplace=True)

# Add to ModifiedName
for c in rename_2.keys():
    data_dict.loc[data_dict['PatientInfoColumn'] == c,
                  'ModifiedName'] = rename_2[c]
# End of adding

master_df['Female'] = master_df['Female'].apply(
    lambda x: 1 if x == 'Female' else 0 if x == 'Male' else np.nan)

master_df['MarriedMale'] = (master_df[['Female', 'MarriedMale']]
                            ).apply(lambda x: 1 if x[0] == 0 and x[1] == 'Married' else 0, axis=1)

master_df['RacialMinority'] = (master_df['RacialMinority']
                               ).apply(lambda x: 0 if x == 'White' else
                                       np.nan if pd.isna(x) else 1)

master_df['SexOrientationMinority'] = (master_df['SexOrientationMinority']
                                       ).apply(lambda x: 0 if x == 'Heterosexual / Straight' or x == 0 else np.nan if pd.isna(x) else 1)
agn_or_ath = 'AgnosticOrAtheist'
master_df.insert(loc=master_df.columns.get_loc('ReligiousMinority'),
                 column=agn_or_ath,
                 value=(
    master_df['ReligiousMinority'].apply(
        lambda x: 1 if x == 'Agnostic' or x == 'Atheist' else np.nan if pd.isna(x) else 0)
))

# Add to ModifiedName
index = data_dict.loc[data_dict['PatientInfoColumn'] == 'Religion'].index[0] + 1
if data_dict.at[index, 'ModifiedName'] != agn_or_ath:
    data_dict.loc[index, 'ModifiedName'] = agn_or_ath
# End of adding

data_dict.to_csv(os.path.join(*data_path, 'DataDictionary.csv'), index=False)

master_df['ReligiousMinority'] = master_df['ReligiousMinority'].apply(lambda x: 0 if (
    x == 'Christian' or x == 'Atheist' or x == 'Agnostic') else np.nan if pd.isna(x) else 0)

# Dr. Erekson said 'In what college is you major' might be relevent is the client is
# a nursing or law student
master_df['NursingOrLaw'] = 0  # most client's did not answer this question,
# of those who did, none were nursing or law
# Here is a guess of what the code might look like if there were:
# master['NursingOrLaw'] = master['NursingOrLaw'].apply(lambda x: 1 if x=='Law' or x=='Nursing' else 0)


def to_homeless(x):
    x = str(x)
    if 'homeless' in x.lower() or 'none' in x.lower() or x.lower() == 'no':
        return 1
    else:
        return 0


master_df['Homeless'] = master_df['Homeless'].apply(to_homeless)
# This is a hard one to convert, since it is not multiple choice
# After taking a look as some of the answers, looking for homeless and none,
# somewhere in the string or if the string is simply 'no' seemed like a wise choice


master_df.to_csv(os.path.join(*data_path, 'master_frame2.csv'), index=False)
print('master_frame2.csv created')

master_df = pd.read_csv(os.path.join(*data_path, 'master_frame2.csv'))

print('Columns that are not numeric:')
for c in master_df.columns:
    if master_df.dtypes[c] not in ['int64', 'float64', 'uint8']:
        print('{0:30}\t{1}'.format(c, master_df.dtypes[c]))
# PatientID and TherapistID should be treated as objects

print(dropped)


### Imputation

In [ ]:

master_df = pd.read_csv(os.path.join(*data_path, 'master_frame2.csv'))

# If column has NAs, print how many
print('{0:30}\t{1}'.format('Column', 'Number of NAs'))
for i, j in zip(master_df.columns, master_df.isna().apply(sum, axis=0)):
    if j != 0:
        print('{0:30}\t{1}'.format(i, j))

master_df['PatientID'] = master_df['PatientID'].astype(str)
master_df['TherapistID'] = master_df['TherapistID'].astype(str)

# Fill some NAs

master_df['age'].fillna(master_df['age'].median(), inplace=True)


# For the following columns, there are a sizeable number of NAs, which will be filled with 0
# The assumption is that when these clients skip the question, they answer in the negative
no_answer_zero = ['Sexual Orientation']
no_answer_zero += [c for c in master_df.columns if 'Recent' in c]
no_answer_zero += ['In the past week, about how many nights has it taken you more than half an hour to fall asleep?',
                   'In the past week, about how many nights have you woken during the night AND needed more than half an hour to fall back to sleep?',
                   'In the past two weeks, have you taken a substance to help with sleep? Please consider prescription medication, over the counter medication and supplements, and substances such as alcohol and marijuana.',
                   'In what college is your current major?',
                   'Academics',
                   'Social life/relationships',
                   'Emotional well-being',
                   'Pornography',
                   'ROTC',
                   'Military Stress']
# Pornography, ROTC, and Military Stress had few than 20,000 NAs in master_df but more than 1,000
# The rest had more than 20,000 NAs

# Make dictionary for replacing values
replace = {}
for c in no_answer_zero:
    replace[c] = 0

master_df.fillna(replace, inplace=True)

median_fill = dict(master_df.loc[:, 'Female':].dropna().apply(np.median, axis=0))
temp = master_df.loc[:, 'Female':].fillna(median_fill)
Matrix = temp.to_numpy()
print('Calculating SVD')
U, s, VT = np.linalg.svd(Matrix)


n = 10  # A somewhat arbitrary choice
replace = pd.DataFrame(U[:, :n] @ np.diag(s[:n]) @ VT[:n, :],
                       columns=master_df.loc[:, 'Female':].columns)


def to_answer(x, answers):
    '''Rounds x to the closes number in the list set_values'''
    answers = np.array(answers)
    index = np.argmin(np.abs(answers - x))
    return answers[index]


# Use the replace data frame to fill in NAs
for c in replace.columns:
    a = {'answers': master_df[c].dropna().unique()}
    master_df.loc[master_df[c].isna(), c] = (replace.loc[master_df[c].isna(), c]
                                             ).apply(to_answer, **a)


# When talking with Dr. Davey Erekson, it was suggested that we aggregate
# the follow up questions about disabilities and trauma to be a number count of the yes's

# Disability
# rename column name that ask about disabilities
rename = {'Are you registered, with the office for disability services on this campus, as having a documented and diagnosed disability?':
          'Disabilities'}
master_df.rename(columns=rename, inplace=True)

# Add to ModifiedName
data_dict = pd.read_csv(os.path.join(*data_path, 'DataDictionary.csv'))
data_dict.loc[data_dict['PatientInfoColumn'] == list(rename.keys())[0],
              'ModifiedName'] = rename[list(rename.keys())[0]]
# End of adding

disability = [c for c in master_df.columns if 'If you selected, "Yes"' in c]
disability_values = master_df[disability].apply(lambda x: sum(
    x.dropna()), axis=1)  # adds up the number of disabilities
disability_values[
    (pd.to_numeric(master_df['Disabilities'], errors='coerce').replace(
        {np.nan: 0}) > 0) & (disability_values == 0)
] += 1  # This is included for those who said they had a disability but did not mark the follow up questions
master_df.loc[:, 'Disabilities'] = disability_values
# Assigns number of disabilities to Disabilities column

master_df.drop(disability, axis=1, inplace=True)

# Trauma
# column names that ask about childhood trauma
modified_names = ['ChildhoodTrauma', 'Trauma']
childhood = [c for c in master_df.columns if 'Childhood' in c]
childhood_values = master_df[childhood].apply(lambda x: 1 if sum(x.dropna()) > 0 else 0, axis=1)
master_df.insert(loc=master_df.columns.get_loc(
    childhood[0]), column=modified_names[0], value=childhood_values)


# ChildhoodTrauma column is a yes/no

trauma = [c for c in master_df.columns if (
    'Please select the traumatic event' in c) & ('Childhood' not in c)]
# Column names that are about trauma but not childhood trauma
trauma_values = master_df[trauma].apply(lambda x: sum(x.dropna()), axis=1)
master_df.insert(loc=master_df.columns.get_loc(
    trauma[0]), column=modified_names[1], value=trauma_values)
master_df.loc[:, 'Trauma'] += master_df[childhood].apply(lambda x: sum(x.dropna()), axis=1).values
# Trauma is the number of yes's in the trauma columns, including childhood

# Add to ModifiedName
for i, c in enumerate([childhood[0], trauma[0]]):
    index = data_dict.loc[data_dict['PatientInfoColumn'] == c].index[0]
    if data_dict.at[index, 'ModifiedName'] != modified_names[i]:
        data_dict.loc[index, 'ModifiedName'] = modified_names[i]

data_dict.to_csv(os.path.join(*data_path, 'DataDictionary.csv'), index=False)
# End of adding

master_df.drop(childhood, axis=1, inplace=True)
master_df.drop(trauma, axis=1, inplace=True)

master_df.to_csv(os.path.join(*data_path, 'master_frame3.csv'), index=False)

print('master_frame3.csv created')


### Scores

In [ ]:

# Upload data and drop nonsense data
scores = pd.read_csv(os.path.join(*data_path, "OQ.csv"), low_memory=False)
scores.replace('#VALUE!', np.nan, inplace=True)
scores["CurrentScore"].replace([180, -3000, -1982, -1978, -1967,
                               -1956, -1900, -914, 0], np.nan, inplace=True)
scores.dropna(subset=["ClientID", "CurrentScore"], inplace=True)
master_df = pd.read_csv(os.path.join(*data_path, 'master_frame3.csv'))

master_df['PatientID'] = master_df['PatientID'].astype(int).astype(str)
master_df['StartDate'] = pd.to_datetime(master_df['StartDate'])
master_df['EndDate'] = pd.to_datetime(master_df['EndDate'])

# Convert OQ times to datetime objects
scores.index = pd.to_datetime(scores["AdministrationDate"])


# Add columns to master data frame
master_df[
    ['IncomingSubClinical', 'RawScore', 'NonLogged',
        'SubClinicalScore', 'NetDrop', 'ClinicallySignificantChange',
        'SignificantChangeToSubClinical', 'SignificantDeterioration',
        'PenultOrUltOQ', 'FirstOQ', 'NumberOfOQs']
] = 0
# IncomingSubClincial is a feature of the patient

print('Calculating scores')
for i in master_df.index:
    ProgressBar(i, master_df.index[-1])

    # Pull up client
    client = master_df.loc[i]

    # Gather relevant OQ surveys
    mask = scores["ClientID"] == client["PatientID"]
    OQ_scores = scores[mask]
    Scores = OQ_scores.sort_index().loc[
        str(client['StartDate']):str(client['EndDate'])
    ].groupby(level=0)['CurrentScore'].mean()

    # If two of the first four scores are in the "normal" range,
    # the session is marked with 1 in the columns IncomingSubClinical
    if np.sum(Scores[:4] < 64) >= 2:
        master_df.loc[i, 'IncomingSubClinical'] = 1

    if len(Scores) < 2:
        master_df.loc[i, ['RawScore', 'NonLogged', 'SubClinicalScore',
                          'NetDrop', 'ClinicallySignificantChange',
                          'SignificantChangeToSubClinical',
                          'SignificantDeterioration', 'PenultOrUltOQ',
                          'FirstOQ', 'NumberOfOQs']] = (
            np.nan, np.nan, np.nan,
            np.nan, np.nan,
            np.nan,
            np.nan, Scores.max() if len(Scores) > 0 else np.nan,
            Scores.max() if len(Scores) > 0 else np.nan, len(Scores))
        continue

    # Regression
    # We use a modified scale for our x-axis, a mesh between time
    # and appointment frequency
    y_vals = Scores.to_numpy()
    y_log = np.log(y_vals)
    x_vals = np.array(list(map(lambda x: (x - Scores.index[0]).days, Scores.index)))

    # Calculate slopes (Exercise 6.1) in Volume 3
    x_bar = np.mean(x_vals)
    x_log = np.log(1 + x_vals)
    x_logbar = np.mean(x_log)
    y_bar = np.mean(y_vals)
    ylog_bar = np.mean(y_log)
    m = np.sum(x_log * y_log - x_log * ylog_bar) / np.sum(x_log * x_log - x_log * x_logbar)

    m_NoLog = np.sum(x_vals * y_vals - x_vals * y_bar) / np.sum(x_vals * x_vals - x_vals * x_bar)

    # Other scores
    drop = y_vals[0] - y_vals[-1]
    first_score = y_vals[0]
    sig_change = int(drop >= 14)
    sig_to_sub = int(y_vals[0] > 64 and y_vals[-1] <= 63 and drop >= 14)
    sig_deter = int(drop <= -14)
    sub = int(np.min(y_vals) <= 63)
    penult_or_ult = np.max(y_vals[-2:])
    master_df.loc[i, ['RawScore', 'NonLogged', 'SubClinicalScore',
                      'NetDrop', 'ClinicallySignificantChange',
                      'SignificantChangeToSubClinical',
                      'SignificantDeterioration', 'PenultOrUltOQ',
                      'FirstOQ', 'NumberOfOQs']] = (
        m * -1, m_NoLog * -1, sub,
        drop, sig_change,
        sig_to_sub,
        sig_deter, penult_or_ult,
        first_score, len(y_vals))


master_df.to_csv(os.path.join(*data_path, 'master_frame4.csv'), index=False)

print('master_frame4.csv created')


### Filter

In [ ]:

master_df = pd.read_csv(os.path.join(*data_path, 'master_frame4.csv'))
rows = master_df.shape[0]
master_df = master_df.loc[(master_df['DateDifference'] < 180) & (master_df['DateDifference'] > 0), :]
master_df.to_csv(os.path.join(*data_path, 'master_frame5.csv'), index=False)

print('Survey not within 180 days before', master_df.shape[0] - rows)
print(f"Number of clients in test/train data: {master_df['PatientID'].nunique()}")


### Some demographic info

In [ ]:
master_df = pd.read_csv(os.path.join(*data_path, 'master_frame5.csv'))
dg = master_df.drop_duplicates(subset=['PatientID'], keep='first')
ages = [int(i) for i in np.quantile(dg['age'], [0,.25,.5,.75,1])]
print(f'Female {dg["Female"].mean() * 100:.1f}%')
print(f'Racial minority {dg["RacialMinority"].mean() * 100:.1f}%')
print(f'''Age: 
min \t{ages[0]:}
25th \t{ages[1]}
median \t{ages[2]}
75th \t{ages[3]}
max \t{ages[4]}''')

# Feature Selection

In [ ]:
#Insert code for generating features.txt here

# Model Selection

In [ ]:
#Recreate Chart of comparison of different model MAEs


data_file = "master_frame5.csv"
features_file = "features.txt"
target = "NetDrop"
data_cleaning = False
feature_selection = False
pickle_file_name = "current_model.pkl"




#Read the features file into a list
features_list = []
with open(features_file, 'r') as myfile:
    for line in myfile:
        features_list.append(line.strip())
    
#Check to see the features list was read in properly
if (len(features_list) == 0):
    print("Something went wrong in reading the features file")

#Get the X,y columns to be used in training
try:
    X,y = CAPS.GetXy(X_columns = features_list, y_column = target, dropna = True, csv_file = data_file)
except:
    print("The csv file does not exist")

#Split X and y into training and testing data
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2, random_state = 2021)


model_maes = {}



### Baseline

In [ ]:
y_vals = y_train.to_numpy()
test_vals = y_test.to_numpy()
average_drop = sum(y_vals)/len(y_vals)
dif_from_average = [abs(x-average_drop) for x in test_vals]
squared_dif = [(x-average_drop)**2 for x in test_vals]
print("The mean squared error of the baseline model is:", sum(squared_dif)/len(squared_dif))
print("The mean absolute error of the baseline model is:", sum(dif_from_average)/len(dif_from_average))
model_maes['baseline'] = sum(dif_from_average)/len(dif_from_average)

### Linear SVR

In [ ]:
regressor_config_dict = {
    'sklearn.svm.LinearSVR': {
        'loss': ["epsilon_insensitive", "squared_epsilon_insensitive"],
        'dual': [True, False],
        'tol': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1],
        'C': [1e-4, 1e-3, 1e-2, 1e-1, 0.5, 1., 5., 10., 15., 20., 25.],
        'epsilon': [1e-4, 1e-3, 1e-2, 1e-1, 1.]
    },
}

model = TPOTRegressor(generations=3, population_size=50, scoring='neg_mean_absolute_error', verbosity=2, random_state=1, n_jobs=-1, config_dict=regressor_config_dict)
print("Training the Linear SVR...")
model.fit(X_train, y_train)


#Use the model to predict outcomes and report the accuracy of predictions
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
print('The mean squared error of Linear SVR is: ', mse)
print('The mean absolute error of Linear SVR is: ', mae)
model_maes['linear svr'] = mae

### Lasso Lars

In [ ]:
#Train TPOT on only Lasso Lars
regressor_config_dict = {
    'sklearn.linear_model.LassoLarsCV': {
        'normalize': [True, False]
    },
}

model = TPOTRegressor(generations=3, population_size=50, scoring='neg_mean_absolute_error', verbosity=2, random_state=1, n_jobs=-1, config_dict=regressor_config_dict)
print("Training the Lasso LARS...")
model.fit(X_train, y_train)


#Use the model to predict outcomes and report the accuracy of predictions
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
print('The mean squared error of Lasso LARS is: ', mse)
print('The mean absolute error of Lasso LARS is: ', mae)
model_maes['lasso lars'] = mae

### XGBoost

In [ ]:
#Train TPOT on only XGBoost
regressor_config_dict = {
    'xgboost.XGBRegressor': {
        'n_estimators': [100],
        'max_depth': range(1, 11),
        'learning_rate': [1e-3, 1e-2, 1e-1, 0.5, 1.],
        'subsample': np.arange(0.05, 1.01, 0.05),
        'min_child_weight': range(1, 21),
        'n_jobs': [1],
        'verbosity': [0],
        'objective': ['reg:squarederror']
    },
}

model = TPOTRegressor(generations=3, population_size=50, scoring='neg_mean_absolute_error', verbosity=2, random_state=1, n_jobs=-1, config_dict=regressor_config_dict)
print("Training the XGBoost...")
model.fit(X_train, y_train)


#Use the model to predict outcomes and report the accuracy of predictions
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
print('The mean squared error of XGBoost is: ', mse)
print('The mean absolute error of XGBoost is: ', mae)
model_maes['xgboost'] = mae

### Gradient Boosting

In [ ]:
#Train TPOT on only Gradient Boosting
regressor_config_dict = {
    'sklearn.ensemble.GradientBoostingRegressor': {
        'n_estimators': [100],
        'loss': ["ls", "lad", "huber", "quantile"],
        'learning_rate': [1e-3, 1e-2, 1e-1, 0.5, 1.],
        'max_depth': range(1, 11),
        'min_samples_split': range(2, 21),
        'min_samples_leaf': range(1, 21),
        'subsample': np.arange(0.05, 1.01, 0.05),
        'max_features': np.arange(0.05, 1.01, 0.05),
        'alpha': [0.75, 0.8, 0.85, 0.9, 0.95, 0.99]
    },
}

model = TPOTRegressor(generations=3, population_size=50, scoring='neg_mean_absolute_error', verbosity=2, random_state=1, n_jobs=-1, config_dict=regressor_config_dict)
print("Training the Gradient Boosting model...")
model.fit(X_train, y_train)


#Use the model to predict outcomes and report the accuracy of predictions
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
print('The mean squared error of Gradient Boosting is: ', mse)
print('The mean absolute error of Gradient Boosting is: ', mae)
model_maes['gradient boosting'] = mae

### K Nearest Neighbors

In [ ]:
#Train TPOT on only K Nearest Neighbors
regressor_config_dict = {
    'sklearn.neighbors.KNeighborsRegressor': {
        'n_neighbors': range(1, 101),
        'weights': ["uniform", "distance"],
        'p': [1, 2]
    },
}

model = TPOTRegressor(generations=3, population_size=50, scoring='neg_mean_absolute_error', verbosity=2, random_state=1, n_jobs=-1, config_dict=regressor_config_dict)
print("Training the K Nearest Neighbors model...")
model.fit(X_train, y_train)


#Use the model to predict outcomes and report the accuracy of predictions
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
print('The mean squared error of K Nearest Neighbors is: ', mse)
print('The mean absolute error of K Nearest Neighbors is: ', mae)
model_maes['k neighbors'] = mae

### Random Forest

In [ ]:
#Train TPOT on only Random Forest
regressor_config_dict = {
    'sklearn.ensemble.RandomForestRegressor': {
        'n_estimators': [100],
        'max_features': np.arange(0.05, 1.01, 0.05),
        'min_samples_split': range(2, 21),
        'min_samples_leaf': range(1, 21),
        'bootstrap': [True, False]
    },
}

model = TPOTRegressor(generations=3, population_size=50, scoring='neg_mean_absolute_error', verbosity=2, random_state=1, n_jobs=-1, config_dict=regressor_config_dict)
print("Training the Random Forest model...")
model.fit(X_train, y_train)


#Use the model to predict outcomes and report the accuracy of predictions
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
print('The mean squared error of Random Forest is: ', mse)
print('The mean absolute error of Random Forest is: ', mae)
model_maes['random forest'] = mae

### Extra Trees

In [ ]:
#Train TPOT on only Extra Trees
regressor_config_dict = {
    'sklearn.ensemble.ExtraTreesRegressor': {
        'n_estimators': [100],
        'max_features': np.arange(0.05, 1.01, 0.05),
        'min_samples_split': range(2, 21),
        'min_samples_leaf': range(1, 21),
        'bootstrap': [True, False]
    },
}

model = TPOTRegressor(generations=3, population_size=50, scoring='neg_mean_absolute_error', verbosity=2, random_state=1, n_jobs=-1, config_dict=regressor_config_dict)
print("Training the Extra Trees model...")
model.fit(X_train, y_train)


#Use the model to predict outcomes and report the accuracy of predictions
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
print('The mean squared error of Extra Trees is: ', mse)
print('The mean absolute error of Extra Trees is: ', mae)
model_maes['extra trees'] = mae

### Comparison Graph

In [ ]:
from matplotlib import pyplot as plt
plt.bar(model_maes.keys(), model_maes.values())

### TPOT with all 30 models

In [ ]:
data_file = "master_frame5.csv"
features_file = "features.txt"
target = "NetDrop"
pickle_file_name = "current_model.pkl"



#Read the features file into a list
features_list = []
with open(features_file, 'r') as myfile:
    for line in myfile:
        features_list.append(line.strip())

#Check to see the features list was read in properly
if (len(features_list) == 0):
    print("Something went wrong in reading the features file")
    
#Get the X,y columns to be used in training
try:
    X,y = CAPS.GetXy(X_columns = features_list, y_column = target, dropna = True, csv_file = data_file)
except:
    print("The csv file does not exist")




#Split X and y into training and testing data
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2, random_state = 2021)

#Checkpoint 
print("The size of the training data is:", len(X_train))
print("The size of the testing data is:", len(y_test))

#Have automl choose the model for us
print("Starting the automl. This may take some time...")
model = TPOTRegressor(generations=3, population_size=50, scoring='neg_mean_absolute_error', verbosity=2, random_state=1, n_jobs=-1)
print("The model has been chosen:", model)
print("Training the model...")
model.fit(X_train, y_train)

#Use the model to predict outcomes and report the accuracy of predictions
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
print('The mean squared error is: ', mse)
print('The mean absolute error is: ', mae)


#Save the model to a pickle file for ease of access in the future
model.export('practice_export.py')
print(model.fitted_pipeline_)
pickle.dump(model.fitted_pipeline_, open(pickle_file_name, 'wb'))


### Top model metrics

In [ ]:
files_y = ("master_frame5.csv", "features.txt", "NetDrop", "current_model.pkl")

model, patientID, X,y = CAPS.GetModelPatientXy(input = files_y)

r_2 = CAPS.METRIC(model=model, metric=r2_score, patientID=patientID, X=X, y=y)
mse = CAPS.METRIC(model=model, metric=mean_squared_error, patientID=patientID, X=X, y=y)
mae = CAPS.MAE(model=model, patientID=patientID, X=X, y=y)

print(model.__str__())
print(f'R-squared: {r_2:.4f}\nMAE:       {mae:.2f}\nRMSE:      {mse**.5:.2f}')


# Improvement Distribution
Needs some updating

### Calculating $\sigma_\epsilon^2$

In [ ]:
# Bad estimator
mse = CAPS.METRIC(model=model, metric=mean_squared_error, patientID=patientID, X=X, y=y)


### Improvement.csv

In [ ]:
#

def improvement(model, patientID, X, y, var_e, select_therapists = None, repetitions=5, n_splits=3, random_state=0, fun = max):
    improvement = pd.DataFrame(columns=['y'], index=X.index)
    improve_y = pd.DataFrame(columns=[f"{r}" for r in range(repetitions)], index=X.index)
    improve_median = pd.DataFrame(columns=[f"{r}" for r in range(repetitions)], index=X.index)
    therapists = [c for c in X.columns if 'TherapistID' in c]
    patient_list = patientID.unique()

    if select_therapists is None:
        select_therapists = therapists
    
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    for ii in range(repetitions):
        for train_p, test_p in kf.split(patient_list):
            train_index = patientID[patientID.isin(patient_list[train_p])].index
            test_index = patientID[patientID.isin(patient_list[test_p])].index

            model.fit(X.loc[train_index, :], y[train_index].to_numpy().ravel())
            outcomes = []
            for therapist in select_therapists:
                X_ = X.loc[test_index, :].copy()
                X_[therapists] = 0
                X_[therapist] = 1
                outcomes.append(model.predict(X_))
            optimal = pd.DataFrame(outcomes).apply(fun, axis=0)
            improve_y.loc[test_index, ii] = (optimal - y[test_index].to_numpy().ravel()).values
    improvement.loc[:, 'y'] = improve_y.mean(axis = 1) 
    improvement.loc[:, 'y'] += np.random.normal(scale = var_e**.5, size = (improve_y.shape[0]))
    return improvement


model, patientID, X,y = CAPS.GetModelPatientXy()
imp_0 = improvement(model=model, patientID=patientID, X=X, y=y, 
                    var_e = 0, repetitions=2, random_state=0)
imp_1 = improvement(model=model, patientID=patientID, X=X, y=y, 
                    var_e = mse, repetitions=2, random_state=0)
rand_0 = improvement(model=model, patientID=patientID, X=X, y=y, 
                     var_e = 0, repetitions=2, random_state=0, fun = np.random.choice)
rand_1 = improvement(model=model, patientID=patientID, X=X, y=y, 
                     var_e = mse, repetitions=2, random_state=0, fun = np.random.choice)
#imp.to_csv(os.path.join(*data_path, 'improvement.csv'), index=False)

In [ ]:
rand_0 = improvement(model=model, patientID=patientID, X=X, y=y, var_e = 0, repetitions=2, random_state=0, fun = np.random.choice)
rand_1 = improvement(model=model, patientID=patientID, X=X, y=y, var_e = mse, repetitions=2, random_state=0, fun = np.random.choice)

### $\mathbb{E}$[ $I$ ], $P(I > 0)$, and $P(I > 14)$

In [ ]:
mean_i = imp_0['y'].mean()
mean_r = rand_0['y'].mean()
p_0 = [np.mean(imp_1['y'] > 0), np.mean(imp_0['y'] > 0), np.mean(rand_1['y'] > 0), np.mean(rand_0['y'] > 0)]
p_14 = [np.mean(imp_0['y'] > 14), np.mean(imp_1['y'] > 14), np.mean(rand_0['y'] > 14), np.mean(rand_1['y'] > 14)]

print(f"""Estimates
E(I) = {mean_i:.2f}
P(I > 0) in ({p_0[0]:.3f}, {p_0[1]:.3f})
P(I > 14) in ({p_14[0]:.3f}, {p_14[1]:.3f})

Random therapist
E(I) = {mean_r:.2f}
P(I > 0) in ({p_0[2]:.3f}, {p_0[3]:.3f})
P(I > 14) in ({p_14[2]:.3f}, {p_14[3]:.3f})
""")

print(f"sd {imp_0['y'].std():.0f}, with error added {imp_1['y'].std():.0f}")

In [ ]:
mean_r

### Bootstrapped numbers

In [ ]:
#

def bootstrap(function, model, X, y, repetitions=1000):
    therapists = [c for c in X.columns if 'TherapistID' in c]

    out = []
    for _ in range(repetitions):
        train_i = choices(sorted(y.index), k = len(y))
        test_i = list(set(y.index) - set(train_i))

        model.fit(X.loc[train_i], y.loc[train_i].to_numpy().ravel())

        outcomes = []
        for therapist in therapists:
            X_ = X.loc[test_i].copy()
            X_[therapists] = 0
            X_[therapist] = 1
            outcomes.append(model.predict(X_))
        optimal = pd.DataFrame(outcomes, index=therapists).apply(np.max, axis=0)
        improve = optimal - y.loc[test_i].to_numpy().ravel()
        out.append(function(improve))
    upper = sorted(out)[round(repetitions * .975) -1]
    lower = sorted(out)[round(repetitions * .25) - 1]
    return {'Mean': np.mean(out), '95% CI': (lower, upper), 'Values': out}

print('Improvement greater than 14 point increase:')
fourteen = bootstrap(lambda x: np.mean(x >= 14), trees, X, y)
print("{Mean} {95% CI}".format(**fourteen))
# 

print('Non negative improvement')
nonnegative = bootstrap(lambda x: np.mean(x >= 0), trees, X, y)
print("{Mean} {95% CI}".format(**nonnegative))
# 

### Assesing bias

In [ ]:
#imp = pd.read_csv(os.path.join(*data_path, "improvement.csv"))
imp = imp_1
_, X_m, _ = CAPS.GetPatientXy(y_column='NetDrop',  dropna=True)
imp.index = [ i for i in range(imp.shape[0]) ]
X_m.index = [ i for i in range(X_m.shape[0]) ]

def get_4groups(series):
    q = np.percentile(series, [25, 50, 75])
    groups = pd.DataFrame(columns=['g'], index=series.index)
    groups.loc[(series <= q[0]), 'g'] = 0
    groups.loc[(series >= q[0]) & (series <= q[1]), 'g'] = 1
    groups.loc[(series >= q[1]) & (series <= q[2]), 'g'] = 2
    groups.loc[(series >= q[2]), 'g'] = 3
    return groups


B = pd.concat([X_m.drop([c for c in X_m.columns if 'TherapistID' in c], axis=1), get_4groups(imp['y'])], axis=1)
B['Disability'] = (B['Disabilities'] > 0).astype(int)

B_columns = pd.DataFrame(B.columns, columns=['o'], index=B.columns)

for character in [",", "(", ")", " ", "'", "-", "/", "&",".", "?"]:
    B_columns.index = [c.replace(character, '') for c in B_columns.index]

B.columns = B_columns.index

mean_table = pd.DataFrame(columns=B.drop(['g'], axis=1).columns)
for level, data in B.groupby("g"):
    mean_table.loc[int(level), 'n'] = data.shape[0]
    for c in mean_table.drop("n", axis=1).columns:
        mean_table.loc[int(level), c] =  data[c].mean()

list_c = ["Female",
          "RacialMinority",
          "age",
          "InternationalStudent",
          "SexOrientationMinority",
          "Disability",
          "ReligiousMinority",
          "AgnosticOrAtheist"]

anova_table = pd.DataFrame(columns=['c', 'p'])
for i, c in enumerate(list_c):
    model_anova = smf.ols(f'{c} ~ C(g)', data=B).fit()
    try:
        aov_table = sm.stats.anova_lm(model_anova, typ=2)
        anova_table.loc[i, 'c'] = B_columns.loc[c, 'o']
        anova_table.loc[i, 'p'] = aov_table.iloc[0]['PR(>F)']
    except ValueError:
        continue

anova_table.sort_values("p", inplace=True)


# Benjamini–Yekutieli procedure
def ben_yek(p_sorted):
    p_sorted
    m = len(p_sorted)
    c = np.log(m) + .57721 + (1 / (2*m))
    significant = [ p_k <= (k+1) / (m * c) * 0.05  for k, p_k in enumerate(p_sorted)]
    return significant

anova_table["significant"] = ben_yek(anova_table['p'])
print(anova_table.loc[anova_table["significant"], :])

font = {'family' : 'normal',
        'weight' : 'regular',
        'size'   : 18}

matplotlib.rc('font', **font)

plt.figure(figsize=(6, 6))
for cc in anova_table.loc[anova_table["significant"], "c"]:
    c = B_columns.loc[B_columns['o'] == cc, :].index[0]
    sns.catplot(data=B,
                x='g',
                hue=cc, 
                kind="count",
               palette='Blues')
    plt.xlabel('Quartiles of Improvement')
    plt.xticks([0, 3], labels=['lowest', 'highest'])
    plt.ylabel(cc)
    plt.savefig(f"../figures/mean-{c}.png")
    plt.clf()


### Therapist Effect and number of clients

In [ ]:
#

def print_counter_fact(X, improve, counter_fact, comment, column, mod_func) -> None:
  improve = pd.concat([improve, counter_fact], axis=1)
  improve.columns = ["y", "y_m"]
  print(comment)
  print(col)
  temp = pd.DataFrame(columns=['level', 'improve_hat', 'sd', 'improvement'])
  for i, l in enumerate(X[column].unique()):
      temp.loc[i, 'level'] = l
      temp.loc[i, 'improve_hat'] = improve[X[col]==l][['y_m', "y"]].apply(lambda x: x[0]-x[1], axis=1).mean()
      temp.loc[i, 'sd'] = improve[X[col]==l][['y_m', "y"]].apply(lambda x: x[0]-x[1], axis=1).std()
      temp.loc[i, 'improvement'] = mod_func(l)
  print(temp.sort_values('level'))

col = 'I feel sad all the time'
y_modified = pd.concat([y, X[['TherapistID_451', col]]], axis=1).apply(
    lambda x: x[0] + x[1] * x[2] * 14 / 4, axis=1).to_frame()
# This therapist appears the most in this data set

imp_1 = [improvement(model=trees, X=X, y=y_modified, select_therapists=['TherapistID_451']),
          improvement(model=trees, X=X, y=y, select_therapists=['TherapistID_451'])]

print_counter_fact(X=X, improve=imp_1[1], counter_fact=imp_1[0],
                    comment='Therapist with the most sessions',
                    column=col, mod_func=lambda x: x * 14 / 4)
'''
  level improve_hat        sd improvement
4   0.0    2.117247  1.870019         0.0
2   1.0    3.249249  2.343237         3.5
1   2.0    6.395659  4.207738         7.0
0   3.0    8.182834  5.137098        10.5
3   4.0   12.140104  5.382635        14.0
'''

col = 'I feel disconnected from myself'
print(col)
y_modified = pd.concat([y, X[['TherapistID_451', col]]], axis=1).apply(
    lambda x: x[0] + x[1] * x[2] * 14 / 4, axis=1).to_frame()

imp_2 = [improvement(model=trees, X=X, y=y_modified, select_therapists=['TherapistID_451']),
          improvement(model=trees, X=X, y=y, select_therapists=['TherapistID_451'])]

print_counter_fact(X=X, improve=imp_2[1], counter_fact=imp_2[0],
                    comment='Therapist with the most sessions',
                    column=col, mod_func=lambda x: x * 14 / 4)
'''
level improve_hat        sd improvement
2   0.0    2.056419  3.331124         0.0
3   1.0    5.247705  3.480443         3.5
1   2.0    8.307339  4.037828         7.0
4   3.0    9.411611  4.721901        10.5
0   4.0   11.799676  5.652768        14.0
'''

print('18 therapist with the fewest')
col = 'I feel sad all the time'
low_app_therapists = [f"TherapistID_{id}" for id in [1053, 969, 1023, 1050, 943, 917, 1052, 913, 1070, 972, 1002,
            968, 1025, 1043, 225, 923, 187]] # These therapist have the fewest appointments
            # 43, 1057 were in this list but then excluded because it was giving me errors
summed_therap = X[low_app_therapists].sum(axis=1).to_frame()
summed_therap.columns = ['Therapists']

y_modified = pd.concat([y, 
                        pd.concat([summed_therap, X[col]], axis=1)],
                        axis=1).apply(
    lambda x: x[0] + x[1] * x[2] * 14 / 4, axis=1).to_frame()

imp_3 = [improvement(model=trees, X=X, y=y_modified, select_therapists=low_app_therapists),
          improvement(model=trees, X=X, y=y, select_therapists=low_app_therapists)]

print_counter_fact(X=X, improve=imp_3[1], counter_fact=imp_3[0],
                    comment='Therapist with the most sessions',
                    column=col, mod_func=lambda x: x * 14 / 4)
'''
  level improve_hat        sd improvement
4   0.0    0.724389  0.801883         0.0
2   1.0    0.989333  1.049811         3.5
1   2.0    3.373035  1.855928         7.0
0   3.0    3.760619  2.355769        10.5
3   4.0    3.536887  2.734238        14.0
'''

col = 'I feel disconnected from myself'
y_modified = pd.concat([y, 
                        pd.concat([summed_therap, X[col]], axis=1)],
                        axis=1).apply(
    lambda x: x[0] + x[1] * x[2] * 14 / 4, axis=1).to_frame()

imp_4 = [improvement(model=trees, X=X, y=y_modified, select_therapists=low_app_therapists),
          improvement(model=trees, X=X, y=y, select_therapists=low_app_therapists)]

print_counter_fact(X=X, improve=imp_4[1], counter_fact=imp_4[0],
                    comment='Therapist with the most sessions',
                    column=col, mod_func=lambda x: x * 14 / 4)
'''
  level improve_hat        sd improvement
2   0.0    2.332845  1.710144         0.0
3   1.0    2.772999  1.836964         3.5
1   2.0    3.488588  2.012908         7.0
4   3.0    5.161405  2.524587        10.5
0   4.0    5.165624  3.078474        14.0
'''


### Unequal distribution of chosen therapists

In [ ]:
#Insert code for the chosen one graph

# Batching

In [ ]:
#Insert code for the batching problem

# Streaming

In [ ]:
#Insert code for the streaming problem